In [3]:
import os

In [1]:
%pwd

'c:\\Users\\User\\Desktop\\Pneumonia-Detection-End-To-End\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\User\\Desktop\\Pneumonia-Detection-End-To-End'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
                 
                 self.config = read_yaml(config_filepath)
                 self.params = read_yaml(params_filepath)
                 
                 create_directories([self.config.artifacts_root])
                 
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
           root_dir=config.root_dir,
           source_URL = config.source_URL,
           local_data_file=config.local_data_file,
           unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config
        

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [11]:
# Data Ingestion Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-09-14 21:47:41,439: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-14 21:47:41,441: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-14 21:47:41,443: INFO: common: created directory at: artifacts]
[2024-09-14 21:47:41,445: INFO: common: created directory at: artifacts/data_ingestion]


[2024-09-14 21:47:45,012: INFO: 1297623377: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7901299
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2fe8fb2e17515846b89136afc42d4e4f94808c3c810e80d4dde01d235c6673cd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B2B6:2E9FE5:85D672:9DEF1A:66E593FF
Accept-Ranges: bytes
Date: Sat, 14 Sep 2024 13:47:44 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1231-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1726321664.970805,VS0,VE756
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 2571a078546e4cc01d95aec82959113824f88413
Expires: Sat, 14 Sep 2024 13:52:44 GMT
Source-Age: 0

]
